## Notebook to retrieve IWP from GMI TB and other auxiliary data using QRNN, uses quantNN

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import ipywidgets as w
import matplotlib.pyplot as plt
import numpy as np
import netCDF4
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from torch.utils.data import DataLoader, random_split
from iwc2tb.GMI.gmiData import gmiData
import os
import time
from quantnn.models.pytorch.fully_connected import FullyConnected
from torch import optim
from quantnn.qrnn import QRNN
from torch.optim import Adam
from quantnn.metrics import ScatterPlot
from quantnn.models.pytorch.logging import TensorBoardLogger
from quantnn.transformations import LogLinear, Log

### Set Inputs

In [9]:
quantiles         = np.linspace(0.01, 0.99, 128)

batchSize         = 256


n_layers          = 5
n_neurons         = 256


inputs            = np.array(["ta", "t2m",  "wvp", "z0", "stype"])
ninputs           = len(inputs) + 3 + 10

outputs           = "iwp"

latlims           = [0, 65]

filename          = "qrnn_gmi_adam_sgd.nc"
outputfile        = os.path.join(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/try_training/"), 
                                                    filename)

## read in training data

In [10]:
data = gmiData(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/TB_GMI_train.nc"), 
               inputs, outputs, latlims = latlims,
               batch_size = batchSize)  

n = len(data)
n_train = int(0.9 * n)
n_val = n - n_train

training_data, validation_data = random_split(data, [n_train, n_val])
results = []

print(len(validation_data))

625


In [ ]:
transform         = LogLinear()

skip_connections  = True
batch_norm        = True

skip_connections  = True
batch_norm        = True

model             = FullyConnected(ninputs,
                       quantiles.size,
                       n_layers,
                       n_neurons,
                       skip_connections=skip_connections,
                       batch_norm=batch_norm)


qrnn             = QRNN(quantiles=quantiles, model=model) # no tranform defined

metrics          = ["MeanSquaredError", "Bias", "CRPS", "CalibrationPlot"]

logger = TensorBoardLogger(n_epochs)

# or by directly providing a metric object (If there are configuration parameters to set).
scatter_plot    = ScatterPlot(bins=np.logspace(-2, 2, 21), log_scale=True)
metrics.append(scatter_plot)

for lr, n_epochs in zip([1e-3, 1e-4], [15, 20]):
    
    optimizer = Adam(qrnn.model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)

    qrnn.train(training_data=training_data,
               validation_data=validation_data,
               n_epochs=n_epochs,
               mask=-1,
               device="cpu",
               logger=logger,
               metrics=metrics,
               optimizer=optimizer, 
               scheduler=scheduler,
              );

qrnn.save(outputfile)

Output()

/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/in

                                                                                           
                                                                                           
                                      Training history                                     
                                                                                           
                                                                                           
       Epoch          Training loss      Validation loss      MSE       Bias       CRPS    
 ───────────────────────────────────────────────────────────────────────────────────────── 
     #      LR            Total               Total                                        
                                                                                           
      1   0.0010          0.201               0.192         1.081      0.204     0.378     
      2   0.0010          0.183               0.180         1.050      -0.011    0.347     
      3   0.0010          0.176               0.172         0.898      -0.024    0.336     
      4   0.0009          0.171               0.165         0.885      -0.050    0.321     
      5   0.0008          0.168               0.157         0.766      0.022     0.303     
      6   0.0008          0.164               0.158         0.806      -0.015    0.309     
      7   0.0007          0.159               0.151         0.720      -0.013    0.305     
      8   0.0006          0.158               0.159         0.860      -0.129    0.306     
      9   0.0004          0.153               0.148         0.708      -0.016    0.295     
     10   0.0003          0.150               0.143         0.675      -0.007    0.282     
     11   0.0003          0.148               0.144         0.712      -0.057    0.285     
     12   0.0002          0.143               0.140         0.684      0.012     0.279     
     13   0.0001          0.142               0.141         0.648      -0.008    0.276     
     14   0.0000          0.140               0.141         0.671      -0.007    0.280     
     15   0.0000          0.139               0.138         0.622      -0.003    0.271   

Output()

/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/inderpreet/git/quantnn/quantnn/metrics.py:566: RuntimeWarning: invalid value encountered in true_divide
  img /= norm
/home/in